In [1]:
import numpy as np
from numpy import *
import pandas as pd

from binance.client import Client
from binance.enums import *


import CoreFunctions as cf

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from joblib import dump, load

#You don't need to enter your key/secret in order to get data from the exchange, its only needed for trades in the TradingBot.py class.
api_key = ''
api_secret = ''
client = Client(api_key, api_secret)

candles = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, "01 Jan, 2021", "20 Aou, 2021")
print(candles)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
candles

[[1609459200000,
  '28923.63000000',
  '28961.66000000',
  '28913.12000000',
  '28961.66000000',
  '27.45703200',
  1609459259999,
  '794382.04398665',
  1292,
  '16.77719500',
  '485390.82682460',
  '0'],
 [1609459260000,
  '28961.67000000',
  '29017.50000000',
  '28961.01000000',
  '29009.91000000',
  '58.47750100',
  1609459319999,
  '1695802.89696884',
  1651,
  '33.73381800',
  '978176.46820208',
  '0'],
 [1609459320000,
  '29009.54000000',
  '29016.71000000',
  '28973.58000000',
  '28989.30000000',
  '42.47032900',
  1609459379999,
  '1231358.69059884',
  986,
  '13.24744400',
  '384076.85445305',
  '0'],
 [1609459380000,
  '28989.68000000',
  '28999.85000000',
  '28972.33000000',
  '28982.69000000',
  '30.36067700',
  1609459439999,
  '880016.76348383',
  959,
  '9.45602800',
  '274083.07514154',
  '0'],
 [1609459440000,
  '28982.67000000',
  '28995.93000000',
  '28971.80000000',
  '28975.65000000',
  '24.12433900',
  1609459499999,
  '699226.20560386',
  726,
  '6.81464400',
  

In [208]:
##Convert the raw data from the exchange into a friendlier form with some basic feature creation
x = cf.FeatureCreation(candles)
print(x)

                date      open      high       low     close     volume
0       1.609459e+12  28923.63  28961.66  28961.66  28913.12  27.457032
1       1.609459e+12  28961.67  29017.50  29009.91  28961.01  58.477501
2       1.609459e+12  29009.54  29016.71  28989.30  28973.58  42.470329
3       1.609459e+12  28989.68  28999.85  28982.69  28972.33  30.360677
4       1.609459e+12  28982.67  28995.93  28975.65  28971.80  24.124339
...              ...       ...       ...       ...       ...        ...
331763  1.629417e+12  46680.03  46680.04  46674.78  46651.57  13.686575
331764  1.629417e+12  46671.70  46747.11  46717.13  46665.28  41.449389
331765  1.629417e+12  46717.13  46750.00  46733.29  46717.12  23.284189
331766  1.629418e+12  46733.30  46766.78  46760.62  46733.29  28.574255
331767  1.629418e+12  46760.62  46791.21  46731.44  46720.49  83.757012

[331768 rows x 6 columns]
               o         h         l         c          v       v_c       h_c  \
0       28923.63  28961.66  

In [209]:
x

,o,h,l,c,v,v_c,h_c,o_c,l_c,h_l,...,c_38StepDifference,c_39StepDifference,c_40StepDifference,c_41StepDifference,c_42StepDifference,c_43StepDifference,c_44StepDifference,c_45StepDifference,c_46StepDifference,c_47StepDifference
0,28923.63,28961.66,28961.66,28913.12,27.457032,0.000950,1.001679,1.000364,1.001679,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28961.67,29017.50,29009.91,28961.01,58.477501,0.002019,1.001951,1.000023,1.001688,1.000262,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,29009.54,29016.71,28989.30,28973.58,42.470329,0.001466,1.001489,1.001241,1.000543,1.000946,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,28989.68,28999.85,28982.69,28972.33,30.360677,0.001048,1.000950,1.000599,1.000358,1.000592,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,28982.67,28995.93,28975.65,28971.80,24.124339,0.000833,1.000833,1.000375,1.000133,1.000700,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331763,46680.03,46680.04,46674.78,46651.57,13.686575,0.000293,1.000610,1.000610,1.000498,1.000113,...,0.000628,0.000419,-0.000618,-0.000432,0.000357,0.001697,0.000883,0.000419,0.000523,0.000692
331764,46671.70,46747.11,46717.13,46665.28,41.449389,0.000888,1.001754,1.000138,1.001111,1.000642,...,0.000941,0.000717,0.000508,-0.000528,-0.000342,0.000446,0.001786,0.000972,0.000509,0.000612
331765,46717.13,46750.00,46733.29,46717.12,23.284189,0.000498,1.000704,1.000000,1.000346,1.000358,...,0.001273,0.001755,0.001532,0.001323,0.000287,0.000472,0.001260,0.002600,0.001787,0.001324
331766,46733.30,46766.78,46760.62,46733.29,28.574255,0.000611,1.000717,1.000000,1.000585,1.000132,...,0.000271,0.000510,0.000992,0.000769,0.000560,-0.000475,-0.000290,0.000498,0.001836,0.001023


In [210]:
x['mid'] = (x['l']+x['h'])/2.0

In [211]:
#x = x['mid']

In [213]:
SEQ_LEN = 1  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 1  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "mid"

In [214]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [215]:
x['future'] = x[RATIO_TO_PREDICT].shift(-FUTURE_PERIOD_PREDICT)

In [216]:
x['target2'] = list(map(classify, x[RATIO_TO_PREDICT], x['future']))

In [217]:
##Create our targets
#y = cf.CreateTargets(candles,1)
y : x['target2']
print(y)

[1 1 1 ... 1 1 0]


In [218]:
#y = y[94:]
#y=np.array(y)
y.shape

(331579,)

In [220]:
x = x[94:len(candles)-1]
x.shape

(331673, 127)

In [221]:
x

,o,h,l,c,v,v_c,h_c,o_c,l_c,h_l,...,c_41StepDifference,c_42StepDifference,c_43StepDifference,c_44StepDifference,c_45StepDifference,c_46StepDifference,c_47StepDifference,mid,future,target2
94,29334.88,29371.95,29361.77,29330.94,40.135145,0.001368,1.001398,1.000134,1.001051,1.000347,...,-0.000397,0.000715,0.000470,0.000296,0.000422,0.000073,-0.000609,29366.860,29364.470,0
95,29361.77,29369.02,29359.92,29340.01,37.107711,0.001265,1.000989,1.000742,1.000679,1.000310,...,0.000931,-0.000567,0.000545,0.000300,0.000126,0.000253,-0.000097,29364.470,29411.330,1
96,29358.76,29422.98,29399.68,29358.75,83.937137,0.002859,1.002188,1.000000,1.001394,1.000793,...,0.000737,0.001255,-0.000241,0.000870,0.000625,0.000451,0.000577,29411.330,29423.465,1
97,29399.68,29423.76,29423.17,29398.13,45.712600,0.001555,1.000872,1.000053,1.000852,1.000020,...,0.001163,0.001428,0.001946,0.000451,0.001560,0.001316,0.001142,29423.465,29470.000,1
98,29423.53,29470.00,29470.00,29423.53,168.395262,0.005723,1.001579,1.000000,1.001579,1.000000,...,0.000154,0.000692,0.000957,0.001474,-0.000019,0.001090,0.000845,29470.000,29435.005,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331762,46650.22,46685.21,46680.03,46642.04,16.644892,0.000357,1.000926,1.000175,1.000815,1.000111,...,0.000122,0.001462,0.000647,0.000183,0.000286,0.000456,0.000331,46682.620,46677.410,0
331763,46680.03,46680.04,46674.78,46651.57,13.686575,0.000293,1.000610,1.000610,1.000498,1.000113,...,-0.000432,0.000357,0.001697,0.000883,0.000419,0.000523,0.000692,46677.410,46732.120,1
331764,46671.70,46747.11,46717.13,46665.28,41.449389,0.000888,1.001754,1.000138,1.001111,1.000642,...,-0.000528,-0.000342,0.000446,0.001786,0.000972,0.000509,0.000612,46732.120,46741.645,1
331765,46717.13,46750.00,46733.29,46717.12,23.284189,0.000498,1.000704,1.000000,1.000346,1.000358,...,0.001323,0.000287,0.000472,0.001260,0.002600,0.001787,0.001324,46741.645,46763.700,1


In [267]:
x = x["mid"]

In [268]:
##produce sets, avoiding overlaps!
##data is seporated temporily rather than randomly
##this prevents the model learning stuff it wouldnt know - aka leakage - which can give us false positive models
y_train = y[:200000]
X_train1 = x[:200000]
X_train.shape


(200000, 1, 2)

In [269]:
##Validation set is not used in this starter model, but should be used if using other libraries that support early stopping.
valy = y[200000:300000]
valx1 = x[200000:300000]
valx.shape

(100000, 1, 2)

In [270]:
valy.shape

(100000,)

In [271]:
y_test = y[300000:]
X_test1 = x[300000:]
X_test.shape

(31673, 1, 2)

In [272]:
y_test.shape

(31579,)

In [273]:
from sklearn.preprocessing import MinMaxScaler

In [274]:
# normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(pd.DataFrame(X_train1))
X_test = scaler.fit_transform(pd.DataFrame(X_test1))
valx = scaler.fit_transform(pd.DataFrame(valx1))

In [275]:
# reshape inputs for LSTM [samples, timesteps, features]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
print("Training data shape:", X_train.shape)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print("Test data shape:", X_test.shape)
valx = valx.reshape(valx.shape[0], 1, valx.shape[1])

Training data shape: (200000, 1, 1)
Test data shape: (31673, 1, 1)


In [276]:
# reshape inputs for LSTM [samples, timesteps, features]
#valx = valx.values.reshape(valx.shape[0], 1, valx.shape[1])
print("val data shape:", valx.shape)


val data shape: (100000, 1, 1)


In [277]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
valx =valx.astype(np.float32)

In [278]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, Masking, TimeDistributed
from tensorflow. keras.utils import plot_model
from tensorflow.keras.layers import Bidirectional
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
# Multiple Inputs
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
#from tensorflow.keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Bidirectional

In [279]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [280]:
class GatedActivationUnit(keras.layers.Layer):
    def __init__(self, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)

    def call(self, inputs):
        n_filters = inputs.shape[-1] // 2
        #n_filter2 = inputs.shape[1:]
        linear_output = self.activation(inputs[..., :n_filters])
        gate = keras.activations.sigmoid(inputs[..., n_filters:])
        return self.activation(linear_output) * gate


def wavenet_residual_block(inputs, n_filters, dilation_rate):
    """
    TODO commenter
    """
    z = Conv1D(
        2 * n_filters, kernel_size=1, padding="causal", dilation_rate=dilation_rate
    )(inputs)
    z = GatedActivationUnit()(z)
    #z = Bidirectional(LSTM(n_filters,kernel_initializer="uniform", activation="relu"))(z)
    # `->tanh instead of relu for cudnn optimization.

    z = Bidirectional(LSTM(n_filters, activation="relu", return_sequences=True))(z)
    z = GatedActivationUnit()(z)
    z = Conv1D(n_filters, activation="relu",kernel_size=1)(z)
    #z = tf.keras.layers.Conv2D(n_filter2, activation='relu', dilation_rate=2)(z)
    #z = ODEBlock(n_filters)(z)
    return keras.layers.Add()([z, inputs]), z

In [296]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

n_layers_per_block = 3  # 10 in the paper
n_blocks = 3  # 3 in the paper
n_filters = 32  # 128 in the paper

visible1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
z = Conv1D(filters=32, kernel_size=1, strides=2, padding="causal")(visible1)
skip_to_last = []
for dilation_rate in [2 ** i for i in range(n_layers_per_block)] * n_blocks:
    z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
    skip_to_last.append(skip)
z = keras.activations.relu(keras.layers.Add()(skip_to_last))
pool11 = Dropout(rate=0.2)(z)
pool12 = LSTM(60,kernel_initializer="uniform" ,activation="relu", return_sequences=True)(pool11)
flat1 = TimeDistributed(keras.layers.Dense(X_train.shape[2],activation="sigmoid", ))(pool12)

model = Model(inputs=[visible1], outputs=flat1, name="trading-encoder")
    
print('Model summary', model.summary())

Model: "trading-encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 1)]       0           []                               
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1, 32)        64          ['input_1[0][0]']                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1, 64)        2112        ['conv1d[0][0]']                 
__________________________________________________________________________________________________
gated_activation_unit (GatedAc  (None, 1, 32)        0           ['conv1d_1[0][0]']               
tivationUnit)                                                                       

In [297]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[rounded_accuracy])
history = model.fit(X_train, y_train, epochs=30,batch_size =1000,
                    validation_data=(valx, valy))

Epoch 1/30
200/200 [==============================] - 29s 77ms/step - loss: 0.6908 - rounded_accuracy: 0.5359 - val_loss: 0.6897 - val_rounded_accuracy: 0.5418
Epoch 2/30
200/200 [==============================] - 14s 68ms/step - loss: 0.6906 - rounded_accuracy: 0.5359 - val_loss: 0.6898 - val_rounded_accuracy: 0.5418
Epoch 3/30
200/200 [==============================] - 14s 69ms/step - loss: 0.6906 - rounded_accuracy: 0.5359 - val_loss: 0.6897 - val_rounded_accuracy: 0.5418
Epoch 4/30
200/200 [==============================] - 14s 70ms/step - loss: 0.6906 - rounded_accuracy: 0.5359 - val_loss: 0.6897 - val_rounded_accuracy: 0.5418
Epoch 5/30
200/200 [==============================] - 14s 71ms/step - loss: 0.6906 - rounded_accuracy: 0.5359 - val_loss: 0.6898 - val_rounded_accuracy: 0.5418
Epoch 6/30
200/200 [==============================] - 14s 72ms/step - loss: 0.6906 - rounded_accuracy: 0.5359 - val_loss: 0.6897 - val_rounded_accuracy: 0.5418
Epoch 7/30
200/200 [====================

In [303]:
##Convert the raw data from the exchange into a friendlier form with some basic feature creation
x = cf.FeatureCreation(candles)
print(x)

                date      open      high       low     close     volume
0       1.609459e+12  28923.63  28961.66  28961.66  28913.12  27.457032
1       1.609459e+12  28961.67  29017.50  29009.91  28961.01  58.477501
2       1.609459e+12  29009.54  29016.71  28989.30  28973.58  42.470329
3       1.609459e+12  28989.68  28999.85  28982.69  28972.33  30.360677
4       1.609459e+12  28982.67  28995.93  28975.65  28971.80  24.124339
...              ...       ...       ...       ...       ...        ...
331763  1.629417e+12  46680.03  46680.04  46674.78  46651.57  13.686575
331764  1.629417e+12  46671.70  46747.11  46717.13  46665.28  41.449389
331765  1.629417e+12  46717.13  46750.00  46733.29  46717.12  23.284189
331766  1.629418e+12  46733.30  46766.78  46760.62  46733.29  28.574255
331767  1.629418e+12  46760.62  46791.21  46731.44  46720.49  83.757012

[331768 rows x 6 columns]
               o         h         l         c          v       v_c       h_c  \
0       28923.63  28961.66  

In [324]:
x['mid'] = (x['l']+x['h'])/2.0

In [325]:
SEQ_LEN = 1  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 1  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "mid"

In [326]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [327]:
x['future'] = x[RATIO_TO_PREDICT].shift(-FUTURE_PERIOD_PREDICT)

In [328]:
x['target'] = list(map(classify, x[RATIO_TO_PREDICT], x['future']))

In [329]:
##Create our targets
#y = cf.CreateTargets(candles,1)
y : x['target']
print(y)

[1 1 1 ... 1 1 0]


In [331]:
x.target.value_counts()

0    179812
1    151861
Name: target, dtype: int64

In [310]:
#y = y[94:]
#y=np.array(y)
y.shape

(331579,)

In [311]:
x = x[94:len(candles)-1]
x.shape

(331673, 127)

In [317]:
x = x.drop(columns=["target"])

In [318]:
##produce sets, avoiding overlaps!
##data is seporated temporily rather than randomly
##this prevents the model learning stuff it wouldnt know - aka leakage - which can give us false positive models
y_train = y[:200000]
X_train1 = x[:200000]
X_train.shape

(200000, 1, 1)

In [319]:
##Validation set is not used in this starter model, but should be used if using other libraries that support early stopping.
valy = y[200000:300000]
valx1 = x[200000:300000]
valx.shape

(100000, 1, 1)

In [320]:
y_test = y[300000:]
X_test1 = x[300000:]
X_test1.shape

(31673, 126)

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# create model
#####################################################################################################    
# Constructing the CNN & training phase
# Select the type of the model
model = Sequential()
# Add the first Dense layer with 200 neuron units and ReLu activation function
model.add(Dense(units=200,
            input_dim=126,
            kernel_initializer='uniform',
            activation='relu'))
# Add Dropout to increase or decrease depending on the number of neurons and the data
model.add(Dropout(0.5))
# Add the second Dense layer with 200 neuron units and ReLu activation function
model.add(Dense(units=200,
            kernel_initializer='uniform',
            activation='relu'))
model.add(Dropout(rate=0.5))
# Repeate Vector
model.add(RepeatVector(X_train.shape[1]))
# Temporal long time memory layer 20 neurone
model.add(LSTM(20, return_sequences=True))
# Add Dropout to prevent overfitting
model.add(Dropout(0.5))
#Flattens
model.add(Flatten())
# Add the second Dense layer with 1 neuron units and Sigmoid activation function
model.add(Dense(units=1,
            kernel_initializer='uniform',
            activation='sigmoid'))


print('Model summary', model.summary())
# leaning rate optimizer exponentialDecay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate=1e-2,
decay_steps=10000,
decay_rate=0.9)
#optimizer Adam low learning rate O,1 and exponentiel
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# optimizer Adam  UpPer
opt = tf.keras.optimizers.Adam(learning_rate=0.09)
model.compile(loss="mse", optimizer=optimizer, metrics=[rounded_accuracy])
model.fit(X_train1, y_train, epochs=30, batch_size=10, validation_data=(valx1, valy), shuffle=False)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               25400     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
repeat_vector (RepeatVector  (None, 1, 200)            0         
)                                                                
_________________________________________________________________
lstm (LSTM)                  (None, 1, 20)             17680     
________________________________________________________